In [136]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from nltk import pos_tag, word_tokenize

%matplotlib inline

In [137]:
#wine_base = pd.read_csv("Our_dataset/StemmedWord2vecTop3_parsed_weather_labeled.csv", index_col=0)
wine_base = pd.read_csv("Our_dataset/winemag-data-130k-v2.csv", index_col=0)
wine_base = wine_base.reset_index()
wine_base= wine_base[pd.notnull(wine_base['description'])]
wine_base.head(2)

,index,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


In [138]:
wine_base['description'] = wine_base['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [139]:
#Preprocessing

#replace unknow price with mean
avgPriceWinery = wine_base.groupby("winery")#.price.mean()
wine_base['price'] = wine_base.apply(lambda row: avgPriceWinery.get_group(row["winery"]).price.mean() 
                          if pd.isnull(row['price']) else row['price'],axis=1)
wine_base["price"].fillna(wine_base["price"].mean(),inplace=True)

#delete small country
#actualData = data.groupby("country").filter(lambda x: len(x)>600) 



In [143]:
#delete non and ajective
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

print("1")
taggedDescriptions = []
for idx, row in wine_base.iterrows():
    desc = row["description"]
    tokens = word_tokenize(desc)
    tagged = pos_tag(tokens)
    taggedDescriptions.append(tagged)

print("2")
descriptions = []
for item in taggedDescriptions:
    processed = ""
    for el in item:
        if el[1].startswith("N") or el[1].startswith("J"):  #se iniziano con N o J sono nomi o aggettivi, quello che ci serve di più
            processed = processed + " " + el[0]
    descriptions.append(processed)

wine_base["description"] = descriptions

[nltk_data] Downloading package punkt to /home/simone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/simone/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [144]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)
for index, row in wine_base.iterrows():
    print (row["description"])

 aromas tropical fruit broom brimstone herb palate expressive unripened apple citrus dried sage brisk acidity
 ripe fruity wine firm tannins juicy red berry fruits acidity drinkable
 tart snappy flavors lime flesh dominate green pineapple pokes crisp acidity flavors wine stainless-steel
 pineapple rind lemon pith orange blossom aromas palate opulent notes honey-drizzled guava mango way astringent semidry finish
 much regular bottling rough tannic rustic earthy herbal characteristics unfussy country wine good companion hearty winter stew
 blackberry raspberry typical navarran whiff green herbs case horseradish mouth full tomatoey acidity spicy herbal flavors dark plum fruit finish fresh grabby
 bright informal aromas candied berry white pepper savory herb palate fresh acidity soft tannins
 dry restrained wine offers profusion acidity firm texture much food
 savory thyme notes accent sunnier flavors preserved peach brisk off-dry wine fruity fresh elegant footprint
 great depth flavor fre

 tart fruit melon orange lemon zest bit lime plenty acid stainless full malolactic
 grilled herb oak espresso whiff wet animal fur palate offers cherry game anisette green note bitter sage astringent tannins
 aromas toasted oak coffee bean whiff sawdust bracing palate offers cherry espresso mocha astringent tannins tannins year fruit
 gold mature pg expectations courtesy waxy baked-apple aromas full balanced mouthfeel apple apricot burnt orange flavors due acidity
 green beans snap peas sauvignon green cast overall citrus gooseberry fruit clean finish
 blend fernão pires arinto crisp fruity acidity acidity citrus pineapple flavors drink
 aromas citrus petrol juicy palate tangy acidity semisharp flavors peach tangerine stone-fruit pits fresh citrusy length lingering note orange finish
 spain graciano grape attractive fruity tejo region example ripe layers acidity generous texture soft smooth ready
 medium-bodied wine aromas green apple stone flavors citrusy lime grapefruit silky texture

 aroma cured meat nose recalls italian deli something cal-ital varieties super tannic palate wine bitter underripe
 taste vanilla-sugar palate acidity attention wine off-kilter presence oak finish theme tart
 gsm blend soft flat dull fruit wash chocolate
 nose closed reveals whiffs dark-skinned fruit diluted palate offers raw red cherry fleeting tannins
 fresh blackberry fruits smooth wine rich attractive fresh acidity density fruit soft tannins back chocolate character
 wine upward progression clerc-milon density sweetness delicious acidity complete wine
 dense ripe fruit austere serious edge wine firm juicy blackberry fruits concentrated tannins firm dark potential good red berry fruits
 soft sweet melted dessert pastry explodes blackberry cherry pie flavors smoky toast oak barrels defines deliciousness tannins soft % cabernet next years freshness
 tinata sultry blend syrah grenache rhône-style blend other wine area grapes little-known region capalbio southern part tuscany intense ar

 cab franc rich ripe red black currants cherries mocha sweet earthiness buttered cinnamon toast slight caramelization dry smooth tannins best acidity minerality structure now–2015
 ripe jammy black-fruit sensations nose mouth wine lush expressive notes blackberry black raspberry boysenberry preserves texture crushed satin medium-plus fine tannins cocoa-kissed finish enjoyable additional complexities
 cabernet high marks sheer lusciousness ripe blackberries black cherries currants veneer sweet oak longterm ager next years
 concentrated firm tannic wine feel chiroubles highest vineyards beaujolais black fruits dark tannins concentrated texture age years ready
 mature notes age years mellow texture cherry pie cola spice flavors feels lush delightful fruit
 blackberry cassis aromas varietal scents herbs cab franc feels laudable structure vital acidity flavors leafy spicy berry plum finish peppery notes jolt salinity heft
 vibrant expressive upfront purple-flower fragrances violet mixed fru

 nero d'avola juicy opens ripe black-skinned berry mediterranean scrub aromas fine-grained tannins blackberry jam sage vanilla flavors
 aromas apple cider peach oak lead full palate flavors melon banana nutty quality stable finish tropical fruit honeyed flavors drink
 juicy pomegranate cherry notes bourbon barrel scents hold midpalate flavors finish year bottle
 bonarda everyday red house pour issues solid berry aromas woody bit earthy bouncy fresh palate cheerful flavors plum blackberry cassis spices toasty forward-moving finish
 perfumes touriga nacional dominate soft smooth wine layer tannins structure ripeness fruit delicious full-bodied wine
 grapes ripe smooth wine green fruits acidity balance generous texture wine crispness end tight aftertaste drink
 non vintage white blend % auxerrois % pinot blanc % pinot gris green apple lemon pith aromas lemon curd apple juice pastry flavors creamy finish
 great deal restraint bottling warmer socal region blackberry black currant loam nose 

...

 ripe jammy aromas red currants berries linear-feeling palate zesty core acidity blend carignan garnacha mataro tastes snappy saucy red-berry fruits finish slings bit tomatoey tang
 aromas present notes generous barrel spices herb dark fruit pencil lead dark-cherry flavors smooth supple fine-grained tannins
 syrah two-thirds blend dark color aromas generous french-oak spices mineral olive dark fruit spicy top note palate rich dense full cherry flavors finish
 best vintage date wine sharp well-defined varietal character tight tart lemon orange cherry fruit well-built finish new oak neutral half twice-used barrels highlights toast mocha latte nice touch tannins
 light color depth clarity wine dusty black cherry pomegranate flavors ethereal texture tension earthy acidity clove orange zest touches long finish
 floral crisp acidity exuberant expression variety ample acidiy cherry orange flavors palate brisk finish spicy nuance
 bottling vintner bill foley mother apricot gardenia smoke aroma

 juicy fruity wine estate knights templar full red berry flavors acidity light touch layer tannins signs wood likely age
 soft fruity style darkly wine brims complex flavors blackberries red licorice prosciutto coffee black pepper sandalwood steaks chops stews
 medium-bodied wine conveys fragrances yellow flowers exotic fruit palate offers apple pear white peach alongside fresh acidity almond note
 ripe blackberry beef jerky cassis flavors strong tannins good foods dry full-bodied fruity red wine bottle couple years
 doubt chardonnay ripeness nose warm bit leaden round mouth tropical flavors orange nectarine finish buttery note residual citrus fruit
 crisp fresh dry rosé champagne red fruits acidity apple touch bitterness wine age few months bottle generous feel
 full extracted style shiraz deep aromas licorice earth tar prunes lovely creamy note plum blackberry flavors soy smoke accents palate wine good length fruit spice flavors
 dense plum-prune coffee chocolate aromas wine ripe dar

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-144-627cca0ea20d>", line 4, in <module>
    print (row["description"])
  File "/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel/iostream.py", line 400, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/ipykernel/iostream.py", line 203, in schedule
    self._event_pipe.send(b'')
  File "/home/simone/Documents/univerista/Magistrale/BDA/venv/lib/python3.6/site-packages/zmq/sugar/socket.py", line 392, in send
    return super(Socket, self).send(data, flags=flags, copy=copy, track=track)
  File "zmq/backend/cython/socket.pyx", line 725, in zmq.backend.cython.socket.Socket.send
  F

KeyboardInterrupt: 

In [145]:
X = wine_base.drop(['points'], axis=1)
Y = wine_base['points'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [146]:
list(X_train)

#list of attribute that you can select to train the model

['index',
 'country',
 'description',
 'designation',
 'price',
 'province',
 'region_1',
 'region_2',
 'taster_name',
 'taster_twitter_handle',
 'title',
 'variety',
 'winery']

In [147]:
#select only CountVectorizer_data or TfidfVectorizer_data o none, but NOT the two togheder
CountVectorizer_data=True
TfidfVectorizer_data=False

#here select the actibute that you want to use during the training
features=['price','vintage','country','region_1','taster_name','winery','variety','province','similarityTop3WinesByVariety', 'pr_5', 'pr_6', 'pr_7', 'pr_8', 'pr_9', 'tas_5', 'tas_6', 'tas_7', 'tas_8', 'tas_9','word_count']
features=[]

In [148]:
if(CountVectorizer_data |TfidfVectorizer_data ):
    
    if(CountVectorizer_data):
        vect = CountVectorizer(min_df=5)
        vect.fit(X_train['description'])
        print("vocabulary size: {}".format(len(vect.vocabulary_)))
        X_train_vectored_cv = vect.transform(X_train['description'])
        X_train_final = X_train_vectored_cv
    else:
        vect = TfidfVectorizer(smooth_idf=True, sublinear_tf=False, analyzer='word',min_df=50, ngram_range=(1, 3))
        vect.fit(X_train['description'])
        print("vocabulary size: {}".format(len(vect.vocabulary_)))
        X_train_vectored_tfidf = vect.transform(X_train['description'])
        X_train_final = X_train_vectored_tfidf
        
    for feature in features:
        X_train_final = hstack((X_train_final,np.array(X_train[feature])[:,None]))
else:
    
    X_train_final = X_train.loc[:,features]
       

vocabulary size: 8934


# Training error

In [149]:
lr = LinearRegression()
lr.fit(X_train_final, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [150]:
y_train_pred = lr.predict(X_train_final)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
rmse

1.806906399748487

# Test error

In [151]:
if(CountVectorizer_data | TfidfVectorizer_data ):
    X_test_final = vect.transform(X_test['description'])
    for feature in features:
        X_test_final = hstack((X_test_final,np.array(X_test[feature])[:,None]))
else:
    X_test_final = X_test.loc[:,features]

y_test_pred = lr.predict(X_test_final)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
rmse

1.969615645008334

In [152]:
print('Coefficients: \n', lr.coef_)


print(len( lr.coef_))

Coefficients: 
 [-2.43309166e-01 -1.22711057e+00  6.75914655e-01 -6.15034009e-01
 -1.74734499e-01 -2.12671625e-01  2.44958041e-01 -1.25088116e-01
  3.57846900e-01  6.77934483e-02  3.40032360e-01 -2.22065250e-02
 -1.64917269e-02  3.79862422e-01 -7.04499444e-01  6.33790562e-01
 -1.55721824e-01 -5.09378099e-01  8.26796543e-01  5.12448815e-01
  6.46709999e-01  9.06538167e-01  7.23364015e-01  7.79318834e-01
  1.22462414e+00  7.44717491e-01 -2.35097852e-01 -9.86083653e-01
 -1.54866841e-01 -4.54408792e-01  8.45170231e-01  1.62657869e-01
 -3.12615277e-01 -2.00039300e-01 -2.74492341e-01  2.34587984e-01
 -2.43828495e-01 -2.57147380e-01  7.09190902e-02  3.27475299e-02
  2.24461833e-01  2.20141308e-01  2.86754760e-01  4.04651179e-01
  8.14176335e-01  7.01949680e-01  2.13873515e+00  8.22622759e-01
  1.68157688e-01  8.04984371e-01 -4.01033327e-02  1.01722565e+00
  1.43170706e+00  2.17861847e+00  3.34363151e-01  8.01210489e-01
 -4.75385159e-01  4.30522013e-01 -2.53229157e-01 -1.26040298e+00
 -3.67935

In [153]:
feature_names = np.array(vect.get_feature_names())
feature_names

array(['000', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100',
       '11', '12', '12th', '13', '130', '13th', '14', '14th', '15',
       '15th', '16', '16th', '17', '17th', '18', '18th', '1960s', '1970s',
       '1990s', '19th', '20', '200', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2020s', '2021', '2022', '2023', '2024', '2025', '2028',
       '2030', '20th', '22', '24', '25', '30', '300', '34', '35', '40',
       '400', '44', '45', '50', '500', '55', '60', '65', '70', '75', '80',
       '800', '85', '86', '87', '88', '89', '90', '900', '91', '92', '93',
       '99', 'aaron', 'abacela', 'abandon', 'abate', 'abbey', 'abbott',
       'abbreviated', 'abeja', 'ability', 'able', 'abound', 'abounds',
       'abouriou', 'above', 'abrasive', 'abrupt', 'abruzzo', 'absence',
       'absent', 'absolute', 'abundance', 'abundant', 'abv', 'acacia',
       'acai', 'accent', 'accented', 'accents', 'acceptable', 

In [154]:
# Sort the coefficients from the model
sorted_coef_index = lr.coef_.argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-140:-1]]))

Smallest Coefs:
['weediness' 'maybe' 'unattractive' 'unfocussed' 'earthquake' 'decline'
 'pepi' 'dullness' 'skunky' 'mars']

Largest Coefs: 
['socks' 'matthias' 'masterpiece' 'flor' 'elixir' 'bouscassé' 'rajat'
 'bas' 'perrier' 'bunches' 'hobbs' 'kathy' 'rancio' 'bendigo' 'gavin'
 'child' 'joey' 'pedace' 'daou' 'oaking' 'link' 'salamino' 'oozes'
 'monopoly' 'tortilla' 'unbelievable' 'cos' 'rubicon' 'commanding'
 'pernand' 'beckmeyer' 'figeac' 'wondrous' '92' 'luis' 'cling' 'lato'
 'breathtaking' 'system' 'peloursin' 'sorbara' 'precipice' 'coveted'
 'stylishness' 'steadfast' 'von' 'créme' 'deutz' 'cates' 'pines' 'upward'
 'chèvre' 'katz' 'allen' 'léoville' 'cerebral' 'connection' 'chase'
 'sample' 'guedes' 'notwithstanding' 'mencia' 'larose' 'allow' 'deserves'
 'ensures' 'boldest' 'understanding' '2030' 'pokes' 'revitalizing'
 'mandarine' 'daffodils' '2020s' 'magdelaine' 'preparation' 'lanson'
 'must' 'playfulness' 'command' 'jarvis' 'blair' 'jeremy' 'cascade'
 'marble' 'spreads' 'demer

In [51]:
#basic feature                                              2.859065103992044
#basic feature + manual wordcount                           2.5488092414557646
#basic feature + word2vect                                  2.7374282971613084
#basic feature + weather                                    2.817176644052167
#basic feature + tf_grouped                                 2.8167889079501047
#basic feature + tf_fullData                                2.820394158650923
#basic feature + tfIdf_grouped                              2.800941096964716
#basic feature + tfIdf_fullData_1                           2.823836138011456        
#basic feature + tfidf                                      1.8287619171940812
#basic feature + sklearn word count                         1.8273230684121524  
#(this show that tfidf and sklearn word counttwo are almost equivalent so i just use one of that)
#since sklean wordcount is the best i check the possible improvement on that

#basic feature + sklearn word count + manual word count     1.8269534098049776    +0.001  +-
#basic feature + sklearn word count + word2vect             1.8175582513592656    +0.01   +-
#basic feature + sklearn word count + weater                1.8143524722884636    +0.01   +-
#basic feature + sklearn word count + weater + word2vect    1.8066107753300489    +0.02   +-
#bac fe + skl word ct + wear + word t + manual word count   1.8065145709633865    +0.02   ++-




My conclusion:


both the custom tfidf and tf grouped get a better result that on the all dataset.
the sklean wordsount and tfidf are way better that the other feature, and they are very similar

both word2vect and weater information add new information on the data,
the word2vect add much more infomration that weather on the basic feature, but in the basic information + word count the improvement is almost the same.

for the manual wordcount as i sensed add much more infomation (compared to word2vect and weather) on the basic feature, but they add very few information on the basic feature + sklean word count, meaning that they close to be a sub set of infomation of the sklean word count.

the best think to do it will be check with a p-test the significance of the improvement obtained.
to check if a 0.02 is significant same for 0.01 0.001

the best will be also to do the same test as i did with another much comples model
to validate it
